### Используемые библиотеки

In [1]:
import pandas as pd
from skopt import BayesSearchCV
from catboost import CatBoostClassifier
from skopt.space import Categorical, Real
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
# считываем обработанные данные
df = pd.read_csv("data.csv")
df_y = df[['IC50, mM', 'CC50, mM', 'SI']].copy()
df_x = df.drop(columns=['IC50, mM', 'CC50, mM', 'SI'])

# добавляем новый признак согласно заданию
median = df_y['SI'].median()
df_y['bool'] = (df_y['SI'] > median).astype(bool)


### Классификация SI (превышение медианного значения выборки)

In [3]:
# датасет для сравнения метрик по каждой из модели
metrics = pd.DataFrame(columns=["Accuracy",
                                "Precision",
                                "Recall",
                                "F1-score",
                                "ROC AUC"]
                                )

In [4]:
# избавимся от лишних признаков, выделим целевую переменную и добавим дополнительный признак
y_data = df_y['bool']
x_data = df_x
x_data['CC50, mM'] = df_y['CC50, mM']

# сократим количество признаков до 50 с помощью Mutual Info
selector = SelectKBest(score_func=mutual_info_regression, k=50)
select_x_data = selector.fit_transform(x_data, y_data)
selected_features = x_data.columns[selector.get_support()]
x_data = pd.DataFrame(select_x_data, columns=selected_features)

# разделим данные на тестовыые и тренировочные
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
print(f'Тренеровочные данные: {X_train.shape}, {y_train.shape}')
print(f'Tестовые данные: {X_test.shape}, {y_test.shape}')

# стандартизируем данные train
scaler = StandardScaler()
tmp_data = scaler.fit_transform(X_train)
scaled_X_train = pd.DataFrame(tmp_data, columns=x_data.columns)

# стандартизируем данные test
tmp_data = scaler.transform(X_test)
scaled_X_test = pd.DataFrame(tmp_data, columns=x_data.columns)

Тренеровочные данные: (772, 50), (772,)
Tестовые данные: (194, 50), (194,)


#### RandomForestClassifier

In [5]:
# определим пространство гиперпараметров для модели RandomForestClassifier
param_space_rfr = {
    'n_estimators': (50, 500),
    'max_depth': (1, 50),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 10),
    'max_features': Categorical(['sqrt', 'log2']),
    'bootstrap': Categorical([True, False])
}

# создадим модель
model = RandomForestClassifier(random_state=42)

# оптимизируем с помощью Байесовского поиска
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=param_space_rfr,
    n_iter=20,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42,
)

bayes_search.fit(scaled_X_train, y_train)

# выведем показатели
roc_auc = round(bayes_search.best_score_, 4)
print(f'ROC AUC = {roc_auc}\n')
print('Лучшие параметры модели:')
for k, v in bayes_search.best_params_.items():
    print(f"{k} = {v}")

# сохраним модель с наилучшими параметрами
rfc_model = bayes_search.best_estimator_


ROC AUC = 0.7142

Лучшие параметры модели:
bootstrap = True
max_depth = 41
max_features = log2
min_samples_leaf = 3
min_samples_split = 12
n_estimators = 174


In [6]:
# расcчитаем y
y_pred = rfc_model.predict(scaled_X_test)

# оценим метрики
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {round(accuracy, 4)}")
print(f"Precision: {round(precision, 4)}")  
print(f"Recall: {round(recall, 4)}")        
print(f"F1-score: {round(f1, 4)}")

# сохраним данные в таблицу
res = {"Accuracy": round(accuracy, 4),
       "Precision": round(precision, 4),
       "Recall": round(recall, 4),
       "F1-score": round(f1, 4),
       "ROC AUC": round(roc_auc, 4)}

metrics.loc[0] = res

Accuracy: 0.6753
Precision: 0.6429
Recall: 0.6923
F1-score: 0.6667


#### GradientBoostingClassifier

In [7]:
# определим пространство гиперпараметров для модели GradientBoostingClassifier
param_space_gbr = {
    'n_estimators': (10, 200),
    'max_depth': (1, 20),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 7),
}

# создадим модель
model = GradientBoostingClassifier(random_state=42)

# оптимизируем с помощью Байесовского поиска
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=param_space_gbr,
    n_iter=20,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42,
)

bayes_search.fit(scaled_X_train, y_train)

# выведем показатели
roc_auc = round(bayes_search.best_score_, 4)
print(f'ROC AUC = {roc_auc}\n')
print('Лучшие параметры модели:')
for k, v in bayes_search.best_params_.items():
    print(f"{k} = {v}")

# сохраним модель с наилучшими параметрами
gbc_model = bayes_search.best_estimator_

ROC AUC = 0.7149

Лучшие параметры модели:
max_depth = 9
min_samples_leaf = 4
min_samples_split = 2
n_estimators = 198


In [8]:
# расcчитаем y
y_pred = gbc_model.predict(scaled_X_test)

# оценим метрики
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {round(accuracy, 4)}")
print(f"Precision: {round(precision, 4)}")  
print(f"Recall: {round(recall, 4)}")        
print(f"F1-score: {round(f1, 4)}")

# сохраним данные в таблицу
res = {"Accuracy": round(accuracy, 4),
       "Precision": round(precision, 4),
       "Recall": round(recall, 4),
       "F1-score": round(f1, 4),
       "ROC AUC": round(roc_auc, 4)}

metrics.loc[1] = res

Accuracy: 0.6804
Precision: 0.6559
Recall: 0.6703
F1-score: 0.663


#### CatBoostClassifier

In [9]:
# определим пространство гиперпараметров для модели CatBoostClassifier
param_space_cbc = {
    'iterations': (100, 1000),
    'depth': (4, 10),
    'grow_policy': Categorical(['SymmetricTree', 'Depthwise', 'Lossguide']),
    'loss_function': Categorical(['Logloss', 'CrossEntropy']),
    'l2_leaf_reg': Real(1, 10),
    'min_data_in_leaf': (1, 50),
    'one_hot_max_size': (2, 10)
}

# создадим модель
model = CatBoostClassifier()

# оптимизируем с помощью Байесовского поиска
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=param_space_cbc,
    n_iter=20,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42,
)

bayes_search.fit(scaled_X_train, y_train)

# выведем показатели
roc_auc = round(bayes_search.best_score_, 4)
print(f'ROC AUC = {roc_auc}\n')
print('Лучшие параметры модели:')
for k, v in bayes_search.best_params_.items():
    print(f"{k} = {v}")

# сохраним модель с наилучшими параметрами
cbc_model = bayes_search.best_estimator_

0:	learn: 0.6848610	total: 59.5ms	remaining: 55.9s
1:	learn: 0.6782961	total: 64.4ms	remaining: 30.2s
0:	learn: 0.6855301	total: 58.1ms	remaining: 54.6s
2:	learn: 0.6708231	total: 70.5ms	remaining: 22s
1:	learn: 0.6783768	total: 63.2ms	remaining: 29.7s
3:	learn: 0.6637331	total: 75.9ms	remaining: 17.8s
2:	learn: 0.6696953	total: 69.2ms	remaining: 21.6s
4:	learn: 0.6576289	total: 81.8ms	remaining: 15.3s
3:	learn: 0.6612682	total: 75.1ms	remaining: 17.6s
4:	learn: 0.6548486	total: 83.5ms	remaining: 15.6s
5:	learn: 0.6492427	total: 92.2ms	remaining: 14.4s
0:	learn: 0.6848021	total: 66.7ms	remaining: 1m 2s
0:	learn: 0.6866283	total: 62.8ms	remaining: 59s
5:	learn: 0.6489515	total: 93.4ms	remaining: 14.5s
6:	learn: 0.6440445	total: 103ms	remaining: 13.8s
1:	learn: 0.6764602	total: 78.4ms	remaining: 36.7s
1:	learn: 0.6797257	total: 71ms	remaining: 33.3s
6:	learn: 0.6437587	total: 101ms	remaining: 13.5s
2:	learn: 0.6701093	total: 87.3ms	remaining: 27.3s
7:	learn: 0.6372334	total: 115ms	remain

In [10]:
# расcчитаем y
y_pred = cbc_model.predict(scaled_X_test)

# оценим метрики
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {round(accuracy, 4)}")
print(f"Precision: {round(precision, 4)}")  
print(f"Recall: {round(recall, 4)}")        
print(f"F1-score: {round(f1, 4)}")

# сохраним данные в таблицу
res = {"Accuracy": round(accuracy, 4),
       "Precision": round(precision, 4),
       "Recall": round(recall, 4),
       "F1-score": round(f1, 4),
       "ROC AUC": round(roc_auc, 4)}

metrics.loc[2] = res

Accuracy: 0.6959
Precision: 0.6739
Recall: 0.6813
F1-score: 0.6776


In [11]:
metrics.index = ['RandomForestClassifier', 'GradientBoostingClassifier', 'CatBoostClassifier']
metrics.head()

,Accuracy,Precision,Recall,F1-score,ROC AUC
RandomForestClassifier,0.6753,0.6429,0.6923,0.6667,0.7142
GradientBoostingClassifier,0.6804,0.6559,0.6703,0.6630,0.7149
CatBoostClassifier,0.6959,0.6739,0.6813,0.6776,0.7270


#### Вывод
Было выполнено преобразование признака SI по бинарному критерию. Критерий оценки состоял из сравнения медианного значения признака SI с текущим значением данных.

Таким образом данные превосзодившие медианное значение SI получали значение True, равные или меньшие получали значение False.

Для булевой целевой переменной SI были дообработанны данные:
* Добавлен дополнительный признак CC50, mM в данные для обучения
* Сокращено количество признаков до 50
* Данные разделены на тестовыые и тренировочные
* Проведена стандартизация тренировочных данных и на её основе стандартизированы тестовые данные

Затем были подобраны параметры и построены три модели:

* RandomForestClassifier c параметрами:
    - bootstrap = True
    - max_depth = 41
    - max_features = log2
    - min_samples_leaf = 3
    - min_samples_split = 12
    - n_estimators = 174

* GradientBoostingClassifier с параметрами:
    - max_depth = 9
    - min_samples_leaf = 4
    - min_samples_split = 2
    - n_estimators = 198

* CatBoostClassifier с параметрами:
    - depth = 5
    - grow_policy = SymmetricTree
    - iterations = 408
    - l2_leaf_reg = 5.864048860435419
    - loss_function = Logloss
    - min_data_in_leaf = 21
    - one_hot_max_size = 9

В качестве метрик для оценки работы модели использовались:
* Accuracy
* Precision
* Recall
* F1-score
* ROC AUC

Наилучшие показатели метрик и при обучение и при валидации на тестовых данных получились у модели CatBoostClassifier.